In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import warnings

sns.set()
rcParams['figure.figsize'] = (20,10)
pd.options.display.max_columns = None
warnings.filterwarnings('ignore')
from pprint import pprint
import json

from transformers import AutoTokenizer, AutoModel, AutoConfig, pipeline
from datasets import load_dataset, Dataset, concatenate_datasets

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-3b")

### CNN_DAILY

In [ ]:
cnn = Dataset.from_json('../../data/translated/cnn_dailymail_30k_samples_len_100_1200_words_translated.json')

In [ ]:
input_template="Cuộc trò chuyện giữa con người và trợ lý AI.\n[|Con người|] Tóm tắt ngắn gọn đoạn văn bản sau đây:\n{article_translated}\n[|AI|] "
def mapper(x):
    return {
        'input': input_template.format(article_translated=x['article_translated']),
        'output': 'Đoạn văn bản đã được tóm tắt:\n' + x['highlights_translated'] + '\n[|Con người|]'
    }

temp_cnn = cnn.shuffle().select(range(15000)).map(mapper,batched=False, remove_columns=['article_translated','highlights_translated'])

In [ ]:
print(temp_cnn[1]['input'] + temp_cnn[1]['output'])

In [ ]:
length = temp_cnn.shuffle().select(range(1000)).map(lambda x: {'len':len(tokenizer.tokenize(x['input'] + x['output']))})
length.to_pandas().hist(bins=100)

In [ ]:
temp_cnn[0]

In [ ]:
temp_cnn.to_json('../../data/training_3_8/cnn_15k_samples.jsonl', orient='records', lines=True)

### DIALOG SUMMARY

In [ ]:
dialog = Dataset.from_json('../../data/translated/dialogsum_10k_samples_len_50_words_translated.json')

In [ ]:
dialog

In [ ]:
input_template="Tóm tắt ngắn gọn đoạn hội thoại sau đây:\n{dialogue_translated}\nĐoạn hội thoại đã được tóm tắt:\n"
def mapper(x):
    return {
        'input': input_template.format(dialogue_translated=x['dialogue_translated']),
        'output': x['summary_translated']
    }

temp_dialog = dialog.shuffle().select(range(10000)).map(mapper,batched=False, remove_columns=['dialogue_translated','summary_translated'])

In [ ]:
print(temp_dialog[1]['input'] + temp_dialog[1]['output'])

In [ ]:
temp_dialog[0]

In [ ]:
temp_dialog.to_json('../../data/training_3_8/dialogsum_10k_samples.jsonl', orient='records', lines=True)

### FAQS

In [ ]:
faqs = Dataset.from_json('../../data/training_31_7/faq_chat_10k.jsonl')

In [ ]:
length = faqs.map(lambda x: {'length':len(tokenizer.tokenize(x['input']))})

In [ ]:
pd.length['length']

In [ ]:
print(faqs.shuffle()[0]['input'])

In [ ]:
text = 'Cuộc trò chuyện giữa con người và trợ lý AI.\n'
text2 = ' Nếu không đủ thông tin để trả lời thì trả lời: Tôi không biết.'
text3 = 'Điều luật liên quan: '
text4 = 'Chỉ được trả lời dựa trên điều luật được cung cấp'
def mapper(x):
    return {
        'input': text,
        'output': x['input'].removeprefix(text).replace(text2,'').replace(text3, 'Điều luật liên quan:\n').replace(text4, 'Chỉ được trả lời dựa trên thông tin nằm trong điều luật được cung cấp') + '\n[|Con người|]',
    }

temp_faqs = faqs.map(mapper, batched=False)

In [ ]:
temp = temp_faqs.shuffle()[0]
temp

In [ ]:
print(temp['input'] + temp['output'])

In [ ]:
temp_faqs.to_json('../../data/training_3_8/faqs.jsonl', orient='records', lines=True)

### SHARE_GPT

In [18]:
share = pd.read_json('/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/original/share_gpt_no_code_conversations_40k.json', orient='records', lines=True)
share.columns = ['input']
share.dropna(inplace=True)
share.reset_index(drop=True, inplace=True)
share = Dataset.from_pandas(share)

In [ ]:
share = share.filter(lambda x: len(tokenizer.tokenize(x["input"])) < 2000)

In [ ]:
share.shuffle().select(range(1000)).map(lambda x: {'len':len(tokenizer.tokenize(x['input']))}).to_pandas().hist(bins=100)

In [19]:
# text = 'Cuộc trò chuyện giữa con người và trợ lý AI.\n'
text = 'The conversation between human and AI assistant.\n'
def mapper(x):
    return {
        'input': text,
        'output': x['input'].removeprefix(text) + '\n[|Con người|]'
    }
share = share.map(mapper, batched=False)

Map:   0%|          | 0/38419 [00:00<?, ? examples/s]

In [20]:
share

Dataset({
    features: ['input', 'output'],
    num_rows: 38419
})

In [22]:
share[0]

{'input': 'The conversation between human and AI assistant.\n',
 'output': '[|Human|] Summarize the main ideas of Jeff Walker\'s Product Launch Formula into bullet points as it pertains to a growth marketing agency implementing these strategies and tactics for their clients...\n[|AI|] Here are the main ideas of Jeff Walker\'s Product Launch Formula that can be applied by a growth marketing agency for their clients:\n\n1. Identify the target audience and their needs: Understand the ideal customer for the product or service,and create a messaging that resonates with them.\n2. Pre-launch: Build anticipation and excitement for the launch by creating buzz,gathering testimonials and case studies,and using social media to create awareness.\n3. Launch: Use a well-crafted launch sequence to maximize sales and conversions. This can include offering bonuses,creating scarcity,and using a deadline to create urgency.\n4. Post-launch: Follow up with customers,gather feedback,and continue to provide v

In [ ]:
share

In [24]:
share.to_json('../../data/training_english/share_gpt_38k.jsonl', orient='records', lines=True)

Creating json from Arrow format:   0%|          | 0/39 [00:00<?, ?ba/s]

295274695

### QUORA

In [ ]:
quora = Dataset.from_json('../../data/translated/quora_chat_data_translated.json')

In [ ]:
print(quora.shuffle()[0]['prompt'])

In [ ]:
quora.shuffle().select(range(1000)).map(lambda x: {'len':len(tokenizer.tokenize(x['prompt']))}).to_pandas().hist(bins=100)

In [ ]:
temp_quora = quora.shuffle().select(range(20000))
temp_quora

In [ ]:
text = 'Cuộc trò chuyện giữa con người và trợ lý AI.\n'
def mapper(x):
    return {
        'input': text,
        'output': x['prompt'].removeprefix(text) + '\n[|Con người|]',
    }

temp_quora = temp_quora.map(mapper, batched=False, remove_columns=['prompt'])

In [ ]:
temp_quora.to_json('../../data/training_3_8/quora_20k_samples.jsonl', orient='records', lines=True)

### OASST

In [25]:
# en_oasst = Dataset.from_json('../../data/translated/en_oasst_translated.json')
# en_oasst = en_oasst.rename_column('text_translated','text')
# vi_oasst = Dataset.from_json('../../data/original/oasst/vi_oasst.json')

ds = Dataset.from_json('/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/original/oasst/en_oasst.json')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /Users/phamhoang1408/.cache/huggingface/datasets/json/default-c507d95be8d43b3a/0.0.0. Subsequent calls will reuse this data.


In [27]:
ds[0]

{'text': 'The conversation between human and AI assistant.\n[|Human|] Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.\n[|AI|] "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, 

In [ ]:
oasst = concatenate_datasets([en_oasst,vi_oasst],axis=0)
oasst

In [ ]:
text = 'Cuộc trò chuyện giữa con người và trợ lý AI.\n'
text = 'The conversation between human and AI assistant.\n'
def mapper(x):
    return {
        'input': text,
        'output': x['text'].removeprefix(text) + '\n[|Con người|]',
    }

temp_oasst = oasst.map(mapper, batched=False, remove_columns=['text'])

In [ ]:
temp_oasst

In [ ]:
temp_oasst[0]

In [ ]:
temp_oasst.to_json('../../data/training_3_8/oasst_21k_samples.jsonl', orient='records', lines=True)

In [ ]:
oasst.shuffle().select(range(1000)).map(lambda x: {'len':len(tokenizer.tokenize(x['text']))}).to_pandas().hist(bins=100)

In [ ]:
print(temp_oasst.shuffle()[0])

In [ ]:
temp_oasst.to_json('../../data/training_3_8/oasst_20k_samples.jsonl', orient='records', lines=True)

### ALPACA CHAT

In [ ]:
alpaca = Dataset.from_json('../../data/training_31_7/alpaca_chat_15k.jsonl')
alpaca

In [ ]:
alpaca

In [ ]:
text = 'Cuộc trò chuyện giữa con người và trợ lý AI.\n'
def mapper(x):
    return {
        'input': text,
        'output': x['input'].removeprefix(text)
    }
temp_alpaca = alpaca.map(mapper, batched=False, remove_columns=['input'])

In [ ]:
temp_alpaca[0]

In [ ]:
temp_alpaca.to_json('../../data/training_3_8/alpaca_chat_15k_samples.jsonl', orient='records', lines=True)

### GPT4 INSTRUCT

In [ ]:
gpt4 = Dataset.from_json('../../data/translated/gpt4_instruct_similarity_0_9_translated.json')
gpt4

In [ ]:
gpt4['instruction_translated']

In [ ]:
template1 = "Cuộc trò chuyện giữa con người và trợ lý AI.\n[|Con người|] {instruction_translated}\n{input_translated}\n[|AI|] "
template2 = "Cuộc trò chuyện giữa con người và trợ lý AI.\n[|Con người|] {instruction_translated}\n[|AI|] "

def mapper(x):
    if x['input_translated'] != '' and x['input_translated'] is not None:
        template = template1.format(
            instruction_translated=x['instruction_translated'], 
            input_translated=x['input_translated']
        )
    else:
        template = template2.format(instruction_translated=x['instruction_translated'])
    return {
        'input': template,
        'output': x['response_translated'] + '\n[|Con người|]'
    }

temp_gpt4 = gpt4.map(mapper,batched=False, remove_columns=gpt4.column_names)

In [ ]:
temp_gpt4

In [ ]:
a = temp_gpt4.shuffle()[0]
print(a['input']+a['output'])

In [ ]:
temp_gpt4.to_json('../../data/training_3_8/gpt4_instruct.jsonl', orient='records', lines=True)

### DOLLY WITH CONTEXT

In [ ]:
dolly = Dataset.from_json('../../data/translated/dolly_with_context_translated.jsonl')
dolly = dolly.remove_columns(['instruction', 'context', 'response', 'category_translated'])

In [ ]:
dolly.shuffle()[0]

In [ ]:
template1 = "Cuộc trò chuyện giữa con người và trợ lý AI.\n[|Con người|] Trả lời câu hỏi dựa vào đoạn văn bản dưới đây. Chỉ được trả lời dựa trên thông tin nằm trong văn bản được cung cấp.\nCâu hỏi: {instruction_translated}\nĐoạn văn bản:\n{context_translated}\n[|AI|] "

def mapper(x):
    ip = template1.format(
        instruction_translated=x['instruction_translated'], 
        context_translated=x['context_translated']
    )
    return {
        'input': ip,
        'output': x['response_translated'] + '\n[|Con người|]'
    }

temp_dolly = dolly.map(mapper,batched=False, remove_columns=dolly.column_names)

In [ ]:
temp = temp_dolly.shuffle()[0]
print(temp['input'] + temp['output'])

Cuộc trò chuyện giữa con người và trợ lý AI.
[|Con người|] Trả lời câu hỏi pháp luật dựa vào những điều luật liên quan dưới đây. Chỉ được trả lời dựa trên thông tin nằm trong điều luật được cung cấp.
Câu hỏi: Phạm tội trong trạng thái tinh thần bị kích động mạnh bị xử lý thế nào?
Điều luật liên quan:
Điều 125. Tội giết người trong trạng thái tinh thần bị kích động mạnh 1. Người nào giết người trong trạng thái tinh thần bị kích động mạnh do hành vi trái pháp luật nghiêm trọng của nạn nhân đối với người đó hoặc đối với người thân thích của người đó, thì bị phạt tù từ 06 tháng đến 03 năm. 2. Phạm tội đối với 02 người trở lên, thì bị phạt tù từ 03 năm đến 07 năm.
[|AI|] Giết người trong trạng thái tinh thần bị kích động mạnh Tội giết người trong trạng thái tinh thần bị kích động mạnh được quy định tại Điều 125 Bộ luật Hình sự 2015 sửa đổi, bổ sung 2017 với mức hình phạt như sau: Khung 1: Người nào giết người trong trạng thái tinh thần bị kích động mạnh do hành vi trái pháp luật nghiêm trọng của nạn nhân đối với người đó hoặc đối với người thân thích của người đó, thì bị phạt tù từ 06 tháng đến 03 năm. Khung 2: Phạm tội đối với 02 người trở lên, thì bị phạt tù từ 03 năm đến 07 năm.
[|Con người|]

In [ ]:
temp_dolly.to_json('../../data/training_3_8/dolly_4k_samples.jsonl', orient='records', lines=True)

### Load all

In [ ]:
from glob import glob
def load_dataset(folder_path, dataset_size=None):
    data = []
    for path in glob(folder_path + "/*.jsonl"):
        ds = Dataset.from_json(path)
        for check in ['input', 'output']:
            if check not in ds.column_names:
                raise ValueError("Dataset must have input, output columns")
        # take only the input and output columns
        ds = ds.select_columns(['input', 'output'])
        data.append(ds)
    if dataset_size is None:
        return concatenate_datasets(data, axis=0).shuffle(seed=42)
    return (
        concatenate_datasets(data, axis=0).shuffle(seed=42).select(range(dataset_size))
    )

def clean(ds):
    def mapper(x):
        for column in ds.column_names:
            x[column] = x[column].replace('[|Human|]','[|Con người|]').replace('\n[|Con người|]\n[|Con người|]', '\n[|Con người|]')
        return x
    def filter(x):
        for column in ds.column_names:
            if x[column] == '' or x[column] is None:
                return False
        return True

    new_ds = ds.map(mapper,batched=False)
    new_ds = new_ds.filter(filter)
    return new_ds        

In [ ]:
ds = load_dataset('../../data/training_3_8')
ds = clean(ds)

In [ ]:
ds

In [ ]:
a = ds.shuffle()[0]
print(a['input'] + a['output'])

In [ ]:
path_to_save = '../../data/temp_training/all_7_8.jsonl'
ds.shuffle().select(range(60000)).to_json(path_to_save, orient='records', lines=True)

In [ ]:
ds.shuffle().select(range(1000)).map(lambda x: {'len':len(tokenizer.tokenize(x['input']+x['output']))}).to_pandas().hist(bins=100)

In [ ]:
df = ds.to_pandas()

In [ ]:
df_length = df.apply(lambda x: len(tokenizer.tokenize(x['input']+x['output'])), axis=1)

In [ ]:
temp_df = df[df_length <= 512].reset_index(drop=True)
temp_df.shape

In [ ]:
final_df = temp_df.sample(60000).reset_index(drop=True)

In [ ]:
ds = Dataset.from_pandas(final_df)

In [ ]:
ds.to_json('../../data/temp_training/final_dataset.jsonl', orient='records', lines=True)

In [ ]:
ds = Dataset.from_json('/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/original/formatted_toolformer-similarity-0.8-dataset.json')

In [ ]:
ds.shuffle()[0]